In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/MyDrive/dataset_.zip' -d '/content/drive/MyDrive'

Archive:  /content/drive/MyDrive/dataset_.zip
   creating: /content/drive/MyDrive/contemporary_images/
 extracting: /content/drive/MyDrive/contemporary_images/C47.png  
 extracting: /content/drive/MyDrive/contemporary_images/C48.png  
 extracting: /content/drive/MyDrive/contemporary_images/C49.png  
 extracting: /content/drive/MyDrive/contemporary_images/C50.png  
 extracting: /content/drive/MyDrive/contemporary_images/C51.png  
 extracting: /content/drive/MyDrive/contemporary_images/Copy of 1.jfif  
 extracting: /content/drive/MyDrive/contemporary_images/Copy of 2.jfif  
 extracting: /content/drive/MyDrive/contemporary_images/Copy of 3.jfif  
 extracting: /content/drive/MyDrive/contemporary_images/Copy of 4.jfif  
 extracting: /content/drive/MyDrive/contemporary_images/Copy of 5.jfif  
 extracting: /content/drive/MyDrive/contemporary_images/Copy of 6.jfif  
 extracting: /content/drive/MyDrive/contemporary_images/Copy of 7.jfif  
 extracting: /content/drive/MyDrive/contemporary_images/

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.utils import save_image
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import torchvision.models as models

# ResNet Block
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.in1 = nn.InstanceNorm2d(out_channels, affine=True)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.in2 = nn.InstanceNorm2d(out_channels, affine=True)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.in1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.in2(out)
        return out + residual

# Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.initial = nn.Conv2d(3, 64, kernel_size=7, padding=3)
        self.initial_in = nn.InstanceNorm2d(64, affine=True)
        self.initial_relu = nn.ReLU(inplace=True)
        self.res_blocks = nn.Sequential(*[ResNetBlock(64, 64) for _ in range(9)])
        self.final = nn.Conv2d(64, 3, kernel_size=7, padding=3)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.initial_relu(self.initial_in(self.initial(x)))
        features = self.res_blocks(x)
        x = self.tanh(self.final(features))
        return x, features

# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(128, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(256, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(512, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        )

    def forward(self, x):
        return self.model(x)

# Dataset class
class SingleFolderDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder = folder
        self.transform = transform
        self.images = [os.path.join(folder, img) for img in os.listdir(folder) if img.endswith(('png', 'jpg', 'jpeg'))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# DataLoader function
def load_data(folder, batch_size=1, num_workers=1):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    dataset = SingleFolderDataset(folder, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# Identity Loss
def identity_loss(real_image, same_image):
    return nn.L1Loss()(real_image, same_image) * 5.0

# Training CycleGAN
def train_cycle_gan(vintage_loader, contemporary_loader, num_epochs=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Initializing models
    Gv2c = Generator().to(device)  # Vintage to Contemporary
    Gc2v = Generator().to(device)  # Contemporary to Vintage
    Dv = Discriminator().to(device)  # Vintage Discriminator
    Dc = Discriminator().to(device)  # Contemporary Discriminator

    # Optimizers
    optimizer_G = optim.Adam(list(Gv2c.parameters()) + list(Gc2v.parameters()), lr=0.0002, betas=(0.5, 0.999))
    optimizer_Dv = optim.Adam(Dv.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_Dc = optim.Adam(Dc.parameters(), lr=0.0002, betas=(0.5, 0.999))

    criterion_GAN = nn.MSELoss()
    criterion_cycle = nn.L1Loss()

    os.makedirs('output', exist_ok=True)

    for epoch in range(num_epochs):
        for i, (vintage_images, contemporary_images) in enumerate(zip(vintage_loader, contemporary_loader)):
            vintage_images = vintage_images.to(device)
            contemporary_images = contemporary_images.to(device)

            # Training Generators
            optimizer_G.zero_grad()

            # Forward cycle
            fake_contemporary = Gv2c(vintage_images)[0]
            reconstructed_vintage = Gc2v(fake_contemporary)[0]
            cycle_loss_vintage = criterion_cycle(reconstructed_vintage, vintage_images) * 10.0

            # Backward cycle
            fake_vintage = Gc2v(contemporary_images)[0]
            reconstructed_contemporary = Gv2c(fake_vintage)[0]
            cycle_loss_contemporary = criterion_cycle(reconstructed_contemporary, contemporary_images) * 10.0

            identity_loss_vintage = identity_loss(Gv2c(vintage_images)[0], vintage_images)
            identity_loss_contemporary = identity_loss(Gc2v(contemporary_images)[0], contemporary_images)

            # GAN loss
            loss_G = criterion_GAN(Dc(fake_contemporary), torch.ones_like(Dc(fake_contemporary))) + \
                      criterion_GAN(Dv(fake_vintage), torch.ones_like(Dv(fake_vintage))) + \
                      cycle_loss_vintage + cycle_loss_contemporary + \
                      identity_loss_vintage + identity_loss_contemporary

            loss_G.backward()
            optimizer_G.step()

            # Training Discriminators
            optimizer_Dv.zero_grad()
            loss_Dv_real = criterion_GAN(Dv(vintage_images), torch.ones_like(Dv(vintage_images)))
            loss_Dv_fake = criterion_GAN(Dv(fake_vintage.detach()), torch.zeros_like(Dv(fake_vintage.detach())))
            loss_Dv = (loss_Dv_real + loss_Dv_fake) * 0.5
            loss_Dv.backward()
            optimizer_Dv.step()

            optimizer_Dc.zero_grad()
            loss_Dc_real = criterion_GAN(Dc(contemporary_images), torch.ones_like(Dc(contemporary_images)))
            loss_Dc_fake = criterion_GAN(Dc(fake_contemporary.detach()), torch.zeros_like(Dc(fake_contemporary.detach())))
            loss_Dc = (loss_Dc_real + loss_Dc_fake) * 0.5
            loss_Dc.backward()
            optimizer_Dc.step()

            if epoch % 10 == 0:
                vutils.save_image(fake_contemporary, f"/content/drive/MyDrive/output/fake_contemporary_epoch{epoch}_img{i}.png", normalize=True)
                vutils.save_image(fake_vintage, f"/content/drive/MyDrive/output/fake_vintage_epoch{epoch}_img{i}.png", normalize=True)
                print(f'Epoch {epoch}, Loss G: {loss_G.item()}, Loss Dv: {loss_Dv.item()}, Loss Dc: {loss_Dc.item()}')

    torch.save(Gv2c.state_dict(), '/content/drive/MyDrive/output/Gv2c.pth')
    torch.save(Gc2v.state_dict(), '/content/drive/MyDrive/output/Gc2v.pth')

# Loading data
vintage_loader = load_data('/content/drive/MyDrive/vintage_images')
contemporary_loader = load_data('/content/drive/MyDrive/contemporary_images')

train_cycle_gan(vintage_loader, contemporary_loader, num_epochs=100)

Using device: cuda
Epoch 0, Loss G: 25.128097534179688, Loss Dv: 0.9373265504837036, Loss Dc: 0.6146196126937866
Epoch 0, Loss G: 23.305248260498047, Loss Dv: 1.1052905321121216, Loss Dc: 1.3422024250030518
Epoch 0, Loss G: 20.68710708618164, Loss Dv: 0.9475713968276978, Loss Dc: 1.2238121032714844
Epoch 0, Loss G: 20.008071899414062, Loss Dv: 2.179077625274658, Loss Dc: 1.2332245111465454
Epoch 0, Loss G: 19.289560317993164, Loss Dv: 0.6205824613571167, Loss Dc: 0.5109186768531799
Epoch 0, Loss G: 19.42674446105957, Loss Dv: 0.3484826982021332, Loss Dc: 0.35983169078826904
Epoch 0, Loss G: 18.724390029907227, Loss Dv: 0.2331598848104477, Loss Dc: 0.3775554597377777
Epoch 0, Loss G: 16.393157958984375, Loss Dv: 0.255127489566803, Loss Dc: 0.2692812383174896
Epoch 0, Loss G: 16.720617294311523, Loss Dv: 0.2987891137599945, Loss Dc: 0.26400917768478394
Epoch 0, Loss G: 16.457225799560547, Loss Dv: 0.1501951664686203, Loss Dc: 0.14457310736179352
Epoch 0, Loss G: 16.16514778137207, Loss D

In [10]:
#cycle final
import torch
import numpy as np
from PIL import Image
import os
import torch.nn as nn
from torchvision.transforms import ToTensor, ToPILImage


# os.makedirs('/content/drive/MyDrive/output2_final', exist_ok=True)
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.in1 = nn.InstanceNorm2d(out_channels, affine=True)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.in2 = nn.InstanceNorm2d(out_channels, affine=True)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.in1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.in2(out)
        return out + residual

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.initial = nn.Conv2d(3, 64, kernel_size=7, padding=3)
        self.initial_in = nn.InstanceNorm2d(64, affine=True)
        self.initial_relu = nn.ReLU(inplace=True)
        self.res_blocks = nn.Sequential(*[ResNetBlock(64, 64) for _ in range(9)])
        self.final = nn.Conv2d(64, 3, kernel_size=7, padding=3)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.initial_relu(self.initial_in(self.initial(x)))
        features = self.res_blocks(x)
        x = self.tanh(self.final(features))
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Gv2c = Generator().to(device)  # Vintage to Contemporary
Gc2v = Generator().to(device)  # Contemporary to Vintage

Gv2c.load_state_dict(torch.load('/content/drive/MyDrive/output/Gv2c.pth', map_location=device))
Gc2v.load_state_dict(torch.load('/content/drive/MyDrive/output/Gc2v.pth', map_location=device))

Gv2c.eval()
Gc2v.eval()


def load_image(image_path, size=None):
    
    image = Image.open(image_path).convert('RGB')
    if size:
        image = image.resize(size)
    return image

def save_image(image, output_path):
  
    image.save(output_path)

def transform_image(image):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    return transform(image).unsqueeze(0).to(device)

def tensor_to_image(tensor):
    tensor = tensor.squeeze().cpu().detach()
    transform = transforms.ToPILImage()
    return transform(tensor)

def apply_cycle_gan_transform(image_path, generator, target_size=(256, 256)):
    image = load_image(image_path, size=target_size)
    input_image = transform_image(image)

    with torch.no_grad():
        output_image = generator(input_image)

    return tensor_to_image(output_image)

base_image_path = '/content/drive/MyDrive/contemporary_images/c2.png'
pattern_image_path = '/content/drive/MyDrive/vintage_images/v32.png'
output_path = '/content/drive/MyDrive/output2_final/new2.png'


transformed_base_image = apply_cycle_gan_transform(base_image_path, Gv2c)
transformed_pattern_image = apply_cycle_gan_transform(pattern_image_path, Gc2v)


base_image_np = np.array(transformed_base_image)
pattern_image_np = np.array(transformed_pattern_image)

def blend_images(base_image_array, pattern_image_array, alpha=0.5):
    base_image_array = base_image_array.astype(float)
    pattern_image_array = pattern_image_array.astype(float)

    # Normalize pattern to be in range [0, 255]
    pattern_normalized = (pattern_image_array / pattern_image_array.max()) * 255
    pattern_normalized = np.clip(pattern_normalized, 0, 255)

    # Blend the pattern with the base image
    blended_image = base_image_array * (1 - alpha) + pattern_normalized * alpha
    blended_image = np.clip(blended_image, 0, 255)

    return Image.fromarray(blended_image.astype('uint8'))

blended_with_pattern_image = blend_images(base_image_np, pattern_image_np)

save_image(blended_with_pattern_image, output_path)


<ipython-input-10-a1da28512c44>:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Gv2c.load_state_dict(torch.load('/content/drive/MyDrive/output/Gv2c.pth', map_location=devi

In [11]:
#cycle f2
import torch
import numpy as np
from PIL import Image, ImageFilter
import os
from torchvision import transforms
from torchvision.utils import save_image

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.initial = nn.Conv2d(3, 64, kernel_size=7, padding=3)
        self.initial_in = nn.InstanceNorm2d(64, affine=True)
        self.initial_relu = nn.ReLU(inplace=True)
        self.res_blocks = nn.Sequential(*[ResNetBlock(64, 64) for _ in range(9)])
        self.final = nn.Conv2d(64, 3, kernel_size=7, padding=3)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.initial_relu(self.initial_in(self.initial(x)))
        features = self.res_blocks(x)
        x = self.tanh(self.final(features))
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Gv2c = Generator().to(device)  # Vintage to Contemporary
Gc2v = Generator().to(device)  # Contemporary to Vintage

Gv2c.load_state_dict(torch.load('/content/drive/MyDrive/output/Gv2c.pth', map_location=device))
Gc2v.load_state_dict(torch.load('/content/drive/MyDrive/output/Gc2v.pth', map_location=device))

Gv2c.eval()
Gc2v.eval()


def load_image(image_path, size=None):
    image = Image.open(image_path).convert('RGB')
    if size:
        image = image.resize(size)
    return image

def save_image(image_array, output_path):
    image = Image.fromarray(image_array.astype('uint8'))
    image.save(output_path)

def transform_image(image, generator):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output_tensor = generator(image_tensor)

    output_image = transforms.ToPILImage()(output_tensor.squeeze().cpu())
    return output_image

def extract_pattern_with_color(image_array, contrast_threshold=0.6):

    image_float = image_array.astype(float) / 255.0


    gray_image = Image.fromarray(image_array).convert('L')
    gray_array = np.array(gray_image) / 255.0
    contrast = (gray_array - gray_array.min()) / (gray_array.max() - gray_array.min())
    mask = contrast > contrast_threshold
    mask = np.expand_dims(mask, axis=-1)
    mask = np.repeat(mask, 3, axis=-1)

    pattern = np.where(mask, image_float, 0)
    return pattern

def blend_pattern_with_base(base_image_path, pattern_image_path, alpha=0.5):

    base_image = load_image(base_image_path)
    pattern_image = load_image(pattern_image_path)

    transformed_base_image = transform_image(base_image, Gv2c)
    transformed_pattern_image = transform_image(pattern_image, Gc2v)

    base_image_array = np.array(transformed_base_image)
    pattern_image_array = np.array(transformed_pattern_image)


    if base_image_array.shape[:2] != pattern_image_array.shape[:2]:
        pattern_image = pattern_image.resize(base_image.size)
        pattern_image_array = np.array(pattern_image)


    pattern = extract_pattern_with_color(pattern_image_array)

    base_image_float = base_image_array.astype(float) / 255.0
    pattern_float = pattern.astype(float)

    blended_image = base_image_float * (1 - alpha) + pattern_float * alpha

    blended_image = np.clip(blended_image * 255, 0, 255)

    return blended_image


base_image_path = '/content/drive/MyDrive/contemporary_images/c2.png'
pattern_image_path = '/content/drive/MyDrive/vintage_images/v32.png'
output_path = '/content/drive/MyDrive/output2_final/nf2.png'


blended_with_pattern_image = blend_pattern_with_base(base_image_path, pattern_image_path, alpha=0.5)

save_image(blended_with_pattern_image, output_path)


<ipython-input-11-d451fb14ccb1>:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Gv2c.load_state_dict(torch.load('/content/drive/MyDrive/output/Gv2c.pth', map_location=devi